https://bcho.tistory.com/1187

https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/tutorials/wide_and_deep/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import os
 
os.chdir('/content/drive/My Drive/Colab Notebooks/wide and deep')
%pwd

In [ ]:
%tensorflow_version 1.x 

In [ ]:
import datetime
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
#raw_data = pd.read_csv('cargolist.cargo_full.csv',encoding='CP949')
train_data = pd.read_csv('train_conti  1230.csv',encoding='CP949')
test_data = pd.read_csv('TEST CONTI DATA 1230.csv',encoding='CP949')

In [ ]:
del train_data['Unnamed: 0']
del train_data['index']
del test_data['Unnamed: 0']
del test_data['index']

In [ ]:
data = train_data.append(test_data)

In [ ]:
COLUMNS = data.columns

CONTINUOUS_COLUMNS=['intMemberPrice']
CATEGORICAL_COLUMNS=['strDwZone', 'dateUpint', 'dateDownint', 'addresscode', 'strBirthcode']

In [ ]:
le = LabelEncoder()
labeltrs = le.fit_transform(data['strTrsNum'])

train_data['label'] = labeltrs[:len(train_data['strTrsNum'])]
train_data.drop('strTrsNum', axis = 1, inplace=True)

test_data['label'] = labeltrs[len(train_data['label']):]
test_data.drop('strTrsNum', axis = 1, inplace=True)

# Wide model feature (categorical)

In [ ]:
CATEGORICAL_COLUMNS

In [ ]:
for f in CATEGORICAL_COLUMNS:
    train_data[f] = train_data[f].astype(str)
    test_data[f] = test_data[f].astype(str)
    data[f] = data[f].astype(str)

In [ ]:
strDwZone = tf.contrib.layers.sparse_column_with_keys(column_name="strDwZone", keys=np.unique(data['strDwZone']))
dateUpint = tf.contrib.layers.sparse_column_with_keys(column_name="dateUpint", keys=np.unique(data['dateUpint']))
dateDownint = tf.contrib.layers.sparse_column_with_keys(column_name="dateDownint", keys=np.unique(data['dateDownint']))
addresscode = tf.contrib.layers.sparse_column_with_keys(column_name="addresscode", keys=np.unique(data['addresscode']))
strBirthcode = tf.contrib.layers.sparse_column_with_keys(column_name="strBirthcode", keys=np.unique(data['strBirthcode']))

# Deep model feature (continuous)

In [ ]:
CONTINUOUS_COLUMNS

In [ ]:
intMemberPrice = tf.contrib.layers.real_valued_column("intMemberPrice")

# wide and deep model

In [ ]:
wide_columns = [strDwZone, dateUpint, dateDownint, addresscode, strBirthcode]

deep_columns = [tf.contrib.layers.embedding_column(strDwZone, dimension=8),
                tf.contrib.layers.embedding_column(dateUpint, dimension=8),
                tf.contrib.layers.embedding_column(dateDownint, dimension=8),
                tf.contrib.layers.embedding_column(addresscode, dimension=8),
                tf.contrib.layers.embedding_column(strBirthcode, dimension=8),
                intMemberPrice]


In [ ]:
import pandas as pd
import urllib

# Define the column names for the data sets.
LABEL_COLUMN = 'label'

# Download the training and test data to temporary files.
# Alternatively, you can download them yourself and change train_file and
# test_file to your own paths.
#train_file = tempfile.NamedTemporaryFile()
#test_file = tempfile.NamedTemporaryFile()
#urllib.urlretrieve("http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.data", train_file.name)
#urllib.urlretrieve("http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

# Read the training and test data sets into Pandas dataframe.
#df_train = pd.read_csv(train_file, names=COLUMNS, skipinitialspace=True)
#df_test = pd.read_csv(test_file, names=COLUMNS, skipinitialspace=True, skiprows=1)
#df_train[LABEL_COLUMN] = (df_train['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
#df_test[LABEL_COLUMN] = (df_test['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)

df_train = train_data
df_test = test_data


def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      dense_shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = dict(list(continuous_cols.items()) + list(categorical_cols.items()))
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)
  #label = tf.constant(keras.utils.to_categorical(df[LABEL_COLUMN]))
  # Returns the feature columns and the label.
  return feature_cols, label

def train_input_fn():
  return input_fn(df_train)

def eval_input_fn():
  return input_fn(df_test)

In [ ]:
df_train[CATEGORICAL_COLUMNS].dtypes

# Main

In [ ]:
m = tf.contrib.learn.DNNLinearCombinedClassifier(
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100, 50],
    n_classes = len(np.unique(data['strTrsNum'])))


In [ ]:
start_time = datetime.datetime.now()

m.fit(input_fn=train_input_fn, steps=10000)
results = m.predict_proba(input_fn=eval_input_fn)

finish_time = datetime.datetime.now()

In [ ]:
### top 10 acc
count=0
for prednum in range(len(df_test['label'])):
    pred = next(results)
    prediction10 = sorted(range(len(pred)),key= lambda i: pred[i])[-10:]

    if df_test['label'][prednum] in prediction10:
        count+=1

print("Top 10 Accuracy : ", count/len(df_test['label']))
print("Run time : ", finish_time - start_time)

Top 10 Accuracy :  0.8867924528301887
Run time :  0:01:58.132032


### *== 아래는 원본 코드 ==*

In [ ]:
### top 1 acc
m.fit(input_fn=train_input_fn, steps=100)
results = m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
  print("%s: %s" % (key, results[key]))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmph2zg99ye/model.ckpt.
INFO:tensorflow:loss = -215666.97, step = 2
INFO:tensorflow:global_step/sec: 41.8418
INFO:tensorflow:loss = -17969465000.0, step = 202 (3.952 sec)
INFO:tensorflow:global_step/sec: 65.2227
INFO:tensorflow:global_step/sec: 120.076
I